## Math 425 final project problem 2

In [27]:
import numpy as np
import numpy.linalg as la

import matplotlib as plt
import vec
from vec import Vec
from vecutil import vec2list
from sympy import Matrix

In [28]:
# Copyright 2013 Philip N. Klein
def read_training_data(fname, D=None):
    """Given a file in appropriate format, and given a set D of features,
    returns the pair (A, b) consisting of
    a P-by-D matrix A and a P-vector b,
    where P is a set of patient identification integers (IDs).

    For each patient ID p,
      - row p of A is the D-vector describing patient p's tissue sample,
      - entry p of b is +1 if patient p's tissue is malignant, and -1 if it is benign.

    The set D of features must be a subset of the features in the data (see text).
    """
    file = open(fname)
    params = ["radius", "texture", "perimeter","area","smoothness","compactness","concavity","concave points","symmetry","fractal dimension"];
    stats = ["(mean)", "(stderr)", "(worst)"]
    feature_labels = set([y+x for x in stats for y in params])
    feature_map = {params[i]+stats[j]:j*len(params)+i for i in range(len(params)) for j in range(len(stats))}
    if D is None: D = feature_labels
    feature_vectors = {}
    #patient_diagnoses = {}
    A = []
    b = []
    for line in file:
        row = line.split(",")
        patient_ID = int(row[0])
        b.append(-1) if row[1] == 'B' else b.append(1)
        feature_vectors[patient_ID] = Vec(D, {f:float(row[feature_map[f]+2]) for f in D})
        A.append(vec2list(feature_vectors[patient_ID]))
    return Matrix(A), Matrix(b)

def classifier(x):
    n,m=np.shape(x)
    for i in range(n):
        for j in range(m):
            if x[i][j]>=0:
                x[i][j]=1
            else:
                x[i][j]=-1
    return x 

def compare_norms(x,y):
    x_norm=la.norm(x)
    y_norm=la.norm(y)
    #display difference
    print("Norm 1:",x_norm," Norm 2:",y_norm)
    print("Difference:",np.abs(x_norm-y_norm),"\n")

def count_incorrect_guesses(b,bhat):
    count=0
    for i in range(np.shape(b)[0]):
            if bhat[i][0]!=b[i][0]:
                print("row:",i,"bhat:",bhat[i][0],"b:",b[i][0])
                count+=1
    return count

In [29]:
# Read Training Data
At,bt=read_training_data('train.data')
At=np.array(At).astype(np.float64)
bt=np.array(bt).astype(np.float64)

# Read Test Data
Av,bv=read_training_data('validate.data')
Av=np.array(Av).astype(np.float64)
bv=np.array(bv).astype(np.float64)

len(bt), len(bv)

(300, 260)

## Method: QR + Least Squares
$A_t = QR$

We want to find $Ax\approx b$

$\hat{x}=R^{-1}Q^Tb$

In [33]:
Q,R=la.qr(At)

xhat=np.matmul(np.matmul(la.inv(R),Q.T),bt)

#Compute bhat
bhat=np.matmul(At,xhat)
print("pre classification")
compare_norms(bt,bhat)
print("post classification")
bhat=classifier(bhat)
compare_norms(bt,bhat)

incorrect_guesses = count_incorrect_guesses(bt,bhat)
print("Number of incorrect guesses:",incorrect_guesses)
print("Accuracy: ", 1 - incorrect_guesses/len(bt))

pre classification
Norm 1: 17.320508075688775  Norm 2: 14.916416762611751
Difference: 2.404091313077023 

post classification
Norm 1: 17.320508075688775  Norm 2: 17.320508075688775
Difference: 0.0 

row: 38 bhat: -1.0 b: 1.0
row: 40 bhat: -1.0 b: 1.0
row: 73 bhat: -1.0 b: 1.0
row: 81 bhat: 1.0 b: -1.0
row: 91 bhat: -1.0 b: 1.0
row: 135 bhat: -1.0 b: 1.0
row: 184 bhat: -1.0 b: 1.0
row: 190 bhat: -1.0 b: 1.0
row: 197 bhat: -1.0 b: 1.0
row: 215 bhat: -1.0 b: 1.0
row: 255 bhat: -1.0 b: 1.0
row: 261 bhat: -1.0 b: 1.0
row: 263 bhat: -1.0 b: 1.0
row: 297 bhat: -1.0 b: 1.0
Number of incorrect guesses: 14
Accuracy:  0.9533333333333334


$\hat{x}=(A^TA)^{-1}A^Tb$

In [34]:
xhat=np.matmul(np.matmul(la.inv(np.matmul(At.T,At)),At.T),bt)
bhat=np.matmul(At,xhat)

print("pre classification")
compare_norms(bt,bhat)
print("post classification")
bhat=classifier(bhat)
compare_norms(bt,bhat)

incorrect_guesses = count_incorrect_guesses(bt,bhat)
print()
print("Number of incorrect guesses:",incorrect_guesses)
print("Accuracy: ", 1 - incorrect_guesses/len(bt))

pre classification
Norm 1: 17.320508075688775  Norm 2: 14.916416762601417
Difference: 2.404091313087358 

post classification
Norm 1: 17.320508075688775  Norm 2: 17.320508075688775
Difference: 0.0 

row: 38 bhat: -1.0 b: 1.0
row: 40 bhat: -1.0 b: 1.0
row: 73 bhat: -1.0 b: 1.0
row: 81 bhat: 1.0 b: -1.0
row: 91 bhat: -1.0 b: 1.0
row: 135 bhat: -1.0 b: 1.0
row: 184 bhat: -1.0 b: 1.0
row: 190 bhat: -1.0 b: 1.0
row: 197 bhat: -1.0 b: 1.0
row: 215 bhat: -1.0 b: 1.0
row: 255 bhat: -1.0 b: 1.0
row: 261 bhat: -1.0 b: 1.0
row: 263 bhat: -1.0 b: 1.0
row: 297 bhat: -1.0 b: 1.0

Number of incorrect guesses: 14
Accuracy:  0.9533333333333334


In [35]:
#Compute bhat
bhat=np.matmul(Av,xhat)

#test norms 
print("pre classification")
compare_norms(bv,bhat)

print("post classification")
bhat=classifier(bhat)
compare_norms(bv,bhat)

incorrect_guesses = count_incorrect_guesses(bv,bhat)
print("Number of incorrect guesses:",incorrect_guesses)
print("Accuracy: ", 1 - incorrect_guesses/len(bv))

pre classification
Norm 1: 16.1245154965971  Norm 2: 13.299651891114907
Difference: 2.824863605482191 

post classification
Norm 1: 16.1245154965971  Norm 2: 16.1245154965971
Difference: 0.0 

row: 29 bhat: -1.0 b: 1.0
row: 47 bhat: 1.0 b: -1.0
row: 113 bhat: 1.0 b: -1.0
row: 155 bhat: 1.0 b: -1.0
row: 165 bhat: 1.0 b: -1.0
row: 171 bhat: 1.0 b: -1.0
row: 214 bhat: -1.0 b: 1.0
row: 241 bhat: 1.0 b: -1.0
Number of incorrect guesses: 8
Accuracy:  0.9692307692307692
